In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [30]:
df = pd.read_csv('barcelona.csv')

In [31]:
le_sector = LabelEncoder()
le_cp = LabelEncoder()

df['Sector_Economic_encoded'] = le_sector.fit_transform(df['Sector_Economic'])
df['Codi_Postal_encoded'] = le_cp.fit_transform(df['Codi_Postal'])

df

,Any,Codi_Postal,Sector_Economic,Valor,Fecha_Hora,t2m,tp,Sector_Economic_encoded,Codi_Postal_encoded
0,2019,8001,Indústria,727,2019-01-01 00:00:00,9.646149,0.000000,0,0
1,2019,8001,Indústria,855,2019-01-01 06:00:00,9.818512,0.000000,0,0
2,2019,8001,Indústria,1175,2019-01-01 12:00:00,14.156647,0.000000,0,0
3,2019,8001,Indústria,1214,2019-01-01 18:00:00,12.701080,0.000000,0,0
4,2019,8001,Residencial,28259,2019-01-01 00:00:00,9.646149,0.000000,2,0
...,...,...,...,...,...,...,...,...,...
1194507,2025,8042,Residencial,31585,2025-02-28 18:00:00,11.339508,0.000000,2,41
1194508,2025,8042,Serveis,6132,2025-02-28 00:00:00,11.016998,0.000000,3,41
1194509,2025,8042,Serveis,12113,2025-02-28 06:00:00,9.402496,0.000007,3,41
1194510,2025,8042,Serveis,12507,2025-02-28 12:00:00,13.221832,0.000004,3,41


In [32]:
df = df[df['Sector_Economic'] != 'No especificat'].copy()
df['Sector_Economic_encoded'] = le_sector.fit_transform(df['Sector_Economic'])
df['Fecha_Hora'] = pd.to_datetime(df['Fecha_Hora'])
fecha_inicio = df['Fecha_Hora'].min()
df['days_since_start'] = (df['Fecha_Hora'] - fecha_inicio).dt.days
df


,Any,Codi_Postal,Sector_Economic,Valor,Fecha_Hora,t2m,tp,Sector_Economic_encoded,Codi_Postal_encoded,days_since_start
0,2019,8001,Indústria,727,2019-01-01 00:00:00,9.646149,0.000000,0,0,0
1,2019,8001,Indústria,855,2019-01-01 06:00:00,9.818512,0.000000,0,0,0
2,2019,8001,Indústria,1175,2019-01-01 12:00:00,14.156647,0.000000,0,0,0
3,2019,8001,Indústria,1214,2019-01-01 18:00:00,12.701080,0.000000,0,0,0
4,2019,8001,Residencial,28259,2019-01-01 00:00:00,9.646149,0.000000,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1194507,2025,8042,Residencial,31585,2025-02-28 18:00:00,11.339508,0.000000,1,41,2250
1194508,2025,8042,Serveis,6132,2025-02-28 00:00:00,11.016998,0.000000,2,41,2250
1194509,2025,8042,Serveis,12113,2025-02-28 06:00:00,9.402496,0.000007,2,41,2250
1194510,2025,8042,Serveis,12507,2025-02-28 12:00:00,13.221832,0.000004,2,41,2250


In [10]:
from autogluon.tabular import TabularPredictor

features = ['t2m', 'tp', 'Sector_Economic_encoded', 'Codi_Postal_encoded', 'Any', 'days_since_start']
X = df[features]
y = df['Valor']

train_data = X.copy()
train_data['Valor'] = y

predictor = TabularPredictor(label='Valor', eval_metric='root_mean_squared_error').fit(
    train_data,
    time_limit=300,              # Limitar tiempo total de entrenamiento a 5 minutos
    presets='best_quality',      # Usa presets de mejor calidad (puedes cambiar a 'medium_quality_faster_train' para más rápido)
    num_bag_folds=2,             # Bagging con 2 folds para evitar sobreajuste, menos que el default para ahorrar tiempo
)

No path specified. Models will be saved in: "AutogluonModels\ag-20250620_204057"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       4.09 GB / 15.31 GB (26.7%)
Disk Space Avail:   317.22 GB / 443.75 GB (71.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=2, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdou

In [12]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preparar el DataFrame de test como lo espera AutoGluon
test_data = X_test.copy()
test_data['Valor'] = y_test

# Evaluar el modelo
results = predictor.evaluate(test_data)

print(results)

{'root_mean_squared_error': np.float64(-8381.098980928138), 'mean_squared_error': -70242816.0, 'mean_absolute_error': -5696.94140625, 'r2': 0.9393965601921082, 'pearsonr': 0.9715310513506933, 'median_absolute_error': np.float64(-3487.72265625)}


In [29]:
import holidays

fechas = pd.date_range(start='2019-01-01', end='2025-12-31')

cat_holidays = holidays.CountryHoliday('ES', subdiv='CT')

df2 = pd.DataFrame({'date': fechas})

# Crear columna 'holiday' (1 si es festivo, 0 si no)
df2['holiday'] = df2['date'].apply(lambda x: 1 if x in cat_holidays else 0)

festivos = ['2020-01-01', '2023-04-07', '2023-05-01', '2023-08-15', '2023-12-25']
print("Checking known holidays:")
print(df[df['date'].isin(pd.to_datetime(festivos))][['date', 'holiday']])

Checking known holidays:
           date  holiday
365  2020-01-01        1
1557 2023-04-07        1
1581 2023-05-01        1
1687 2023-08-15        1
1819 2023-12-25        1


In [38]:
df['date'] = df['Fecha_Hora'].dt.date  # Extraemos solo la fecha
df2['date'] = pd.to_datetime(df2['date']).dt.date  # date tipo python
df_merged = df.merge(df2, on='date', how='left')
df_merged


,Any,Codi_Postal,Sector_Economic,Valor,Fecha_Hora,t2m,tp,Sector_Economic_encoded,Codi_Postal_encoded,days_since_start,date,holiday
0,2019,8001,Indústria,727,2019-01-01 00:00:00,9.646149,0.000000,0,0,0,2019-01-01,1
1,2019,8001,Indústria,855,2019-01-01 06:00:00,9.818512,0.000000,0,0,0,2019-01-01,1
2,2019,8001,Indústria,1175,2019-01-01 12:00:00,14.156647,0.000000,0,0,0,2019-01-01,1
3,2019,8001,Indústria,1214,2019-01-01 18:00:00,12.701080,0.000000,0,0,0,2019-01-01,1
4,2019,8001,Residencial,28259,2019-01-01 00:00:00,9.646149,0.000000,1,0,0,2019-01-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1133387,2025,8042,Residencial,31585,2025-02-28 18:00:00,11.339508,0.000000,1,41,2250,2025-02-28,0
1133388,2025,8042,Serveis,6132,2025-02-28 00:00:00,11.016998,0.000000,2,41,2250,2025-02-28,0
1133389,2025,8042,Serveis,12113,2025-02-28 06:00:00,9.402496,0.000007,2,41,2250,2025-02-28,0
1133390,2025,8042,Serveis,12507,2025-02-28 12:00:00,13.221832,0.000004,2,41,2250,2025-02-28,0


In [39]:
features = ['t2m', 'tp', 'Sector_Economic_encoded', 'Codi_Postal_encoded', 'Any', 'days_since_start', 'holiday']
X = df_merged[features]
y = df_merged['Valor']

train_data = X.copy()
train_data['Valor'] = y

predictor = TabularPredictor(label='Valor', eval_metric='root_mean_squared_error').fit(
    train_data,
    time_limit=300,              # Limitar tiempo total de entrenamiento a 5 minutos
    presets='best_quality',      # Usa presets de mejor calidad (puedes cambiar a 'medium_quality_faster_train' para más rápido)
    num_bag_folds=2,             # Bagging con 2 folds para evitar sobreajuste, menos que el default para ahorrar tiempo
)

No path specified. Models will be saved in: "AutogluonModels\ag-20250620_210543"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       2.99 GB / 15.31 GB (19.5%)
Disk Space Avail:   316.66 GB / 443.75 GB (71.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=2, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdou

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preparar el DataFrame de test como lo espera AutoGluon
test_data = X_test.copy()
test_data['Valor'] = y_test

# Evaluar el modelo
results = predictor.evaluate(test_data)

print(results)

{'root_mean_squared_error': np.float64(-8364.3759279738), 'mean_squared_error': -69962784.0, 'mean_absolute_error': -5683.5205078125, 'r2': 0.9396381378173828, 'pearsonr': 0.9716525473587202, 'median_absolute_error': np.float64(-3470.5341796875)}


In [12]:
test_data.to_csv("test_data.csv", index=False)

In [49]:
results

{'root_mean_squared_error': np.float64(-8377.644641263321),
 'mean_squared_error': -70184928.0,
 'mean_absolute_error': -5697.45751953125,
 'r2': 0.9394465088844299,
 'pearsonr': 0.9715547288371824,
 'median_absolute_error': np.float64(-3487.5625)}

In [50]:
predictor.save('./modelo_gasto_temporal')

In [53]:
p = TabularPredictor.load("AutogluonModels/ag-20250618_195525")
p

In [55]:
import pickle 

with open('le_sector.pkl', 'wb') as f:
    pickle.dump(le_sector, f)

with open('le_cp.pkl', 'wb') as f:
    pickle.dump(le_cp, f)

In [77]:
df.iloc[104960]

Any                                       2019
Codi_Postal                               8011
Sector_Economic                        Serveis
Valor                                    63281
Fecha_Hora                 2019-07-28 00:00:00
t2m                                  20.663239
tp                                    0.000137
Sector_Economic_encoded                      2
Codi_Postal_encoded                         10
Mes                                          7
days_since_start                           208
Name: 111696, dtype: object